In [26]:
import pandas as pd 
import tensorflow
import matplotlib

import requests
import tarfile

import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

from tqdm import tqdm    
import random

import matplotlib as mpl
import matplotlib.pyplot as plt

import matplotlib.font_manager as fm

from konlpy.tag import Mecab

# 12-1. 프로젝트: 한영 번역기 만들기

## Step 1. 데이터 다운로드

### 데이터 다운로드

- URL로 다운로드 시도했으나, tar.gz 포맷 인식 못해서 수동으로 다운로드 진행함

### 압축 풀기

In [27]:
# !tar -xvf korean-english-park.train.tar.gz
# !tar -xvf korean-english-park.test.tar.gz
# !tar -xvf korean-english-park.dev.tar.gz

## Step 2. 데이터 정제

set 데이터형이 중복을 허용하지 않는다는 것을 활용해 중복된 데이터를 제거하도록 합니다. 데이터의 병렬 쌍이 흐트러지지 않게 주의하세요! 중복을 제거한 데이터를 cleaned_corpus 에 저장합니다.

앞서 정의한 preprocessing() 함수는 한글에서는 동작하지 않습니다. 한글에 적용할 수 있는 정규식을 추가하여 함수를 재정의하세요!

타겟 언어인 영문엔 <start> 토큰과 <end> 토큰을 추가하고 split() 함수를 이용하여 토큰화합니다. 한글 토큰화는 KoNLPy의 mecab 클래스를 사용합니다.

모든 데이터를 사용할 경우 학습에 굉장히 오랜 시간이 걸립니다. cleaned_corpus로부터 토큰의 길이가 40 이하인 데이터를 선별하여 eng_corpus와 kor_corpus를 각각 구축하세요.

### data load

In [28]:
data = {
    'train': {
        'ko':[],
        'en':[],
    },
    'test': {
        'ko':[],
        'en':[],
    },
    'dev': {
        'ko':[],
        'en':[],
    },
}

# 파일 열기
with open("korean-english-park.train.ko", "r", encoding="utf-8") as file:
    lines = file.read()
    for line in lines.split('\n'):
        data['train']['ko'].append(line)
        
with open("korean-english-park.train.en", "r", encoding="utf-8") as file:
    lines = file.read()
    for line in lines.split('\n'):
        data['train']['en'].append(line)
        
with open("korean-english-park.test.ko", "r", encoding="utf-8") as file:
    lines = file.read()
    for line in lines.split('\n'):
        data['test']['ko'].append(line)
        
with open("korean-english-park.test.en", "r", encoding="utf-8") as file:
    lines = file.read()
    for line in lines.split('\n'):
        data['test']['en'].append(line)
        
with open("korean-english-park.dev.ko", "r", encoding="utf-8") as file:
    lines = file.read()
    for line in lines.split('\n'):
        data['dev']['ko'].append(line)
        
with open("korean-english-park.dev.en", "r", encoding="utf-8") as file:
    lines = file.read()
    for line in lines.split('\n'):
        data['dev']['en'].append(line)


In [29]:
train = pd.DataFrame(
    {
        'ko':data['train']['ko'],
        'en':data['train']['en']
    }
)
test = pd.DataFrame(
    {
        'ko':data['test']['ko'],
        'en':data['test']['en']
    }
)
dev = pd.DataFrame(
    {
        'ko':data['dev']['ko'],
        'en':data['dev']['en']
    }
)

In [30]:
print('train - ko :',len(data['train']['ko']))
print('train - en :',len(data['train']['en']))
print('test - ko :',len(data['test']['ko']))
print('test - en :',len(data['test']['en']))
print('dev - ko :',len(data['dev']['ko']))
print('dev - en :',len(data['dev']['en']))

train - ko : 94124
train - en : 94124
test - ko : 2001
test - en : 2001
dev - ko : 1001
dev - en : 1001


In [31]:
def preprocessing(df):
    print(f'{len(df)} ->' ,end=' ')
    # 중복 제거
    df = df.drop_duplicates(subset=['ko'])
    df = df.drop_duplicates(subset=['en'])
    
    # 정규식
    def preprocess_sentence(sentence):
        sentence = sentence.strip()                                         # 문장의 양쪽 공백 제거
        sentence = re.sub(r"([?.!,])", r" \1 ", sentence)                   # 특수 문자 및 구두점 주변에 공백 추가
        sentence = re.sub(r'[" "]+', " ", sentence)                         # 여러 개의 공백을 하나의 공백으로 대체
        sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z?.!,]+", " ", sentence)  # 한글 및 영어 이외의 문자는 공백으로 대체
        sentence = sentence.strip()                                         # 다시 양쪽 공백 제거
        return sentence
    
    # 'ko'와 'en' 열에 정규식 전처리 함수 적용
    df['ko'] = df['ko'].apply(preprocess_sentence)
    df['en'] = df['en'].apply(preprocess_sentence)
    
    # 길이가 0 인 데이터 제거
    df = df[(df['ko'].str.len() > 0) & (df['en'].str.len() > 0)]
    
    # en 에 <start> , <end> 토큰 추가
    df['en'] = df['en'].apply(lambda x : '<start> '+ x + ' <end>')
    
    # 토큰화 , ko : mecab , en : split
    mecab = Mecab()
    df['ko'] = df['ko'].apply(lambda x : mecab.morphs(x))
    df['en'] = df['en'].apply(lambda x : x.split())
    
    # 토큰의 길이가 20 이하인 데이터를 선별하여 
    df = df[(df['ko'].str.len() < 21) & (df['en'].str.len() < 21)]
    print(f'{len(df)}')
    return df
train = preprocessing(train)
train.head()

94124 -> 13399


,ko,en
0,"[개인, 용, 컴퓨터, 사용, 의, 상당, 부분, 은, 이것, 보다, 뛰어날, 수,...","[<start>, Much, of, personal, computing, is, a..."
2,"[그러나, 이것, 은, 또한, 책상, 도, 필요, 로, 하, 지, 않, 는다, .]","[<start>, Like, all, optical, mice, ,, But, it..."
8,"[결정, 적, 인, 순간, 에, 그, 들, 의, 능력, 을, 증가, 시켜, 줄, 그...","[<start>, Something, that, will, boost, their,..."
15,"[러시아, 특수, 부대, 는, 극장, 으로, 공격, 해, 들어가, 기, 전, 에, ...","[<start>, Russian, special, forces, used, a, s..."
16,"[많, 은, 인질, 들, 이, 화학, 가스, 의, 영향, 으로, 고통, 을, 겪, ...","[<start>, Many, captives, were, taken, to, hos..."


In [32]:
# train = train[:30000]
# print(len(train))

In [33]:
# eng_corpus와 kor_corpus를 각각 구축하세요.
kor_corpus = list(train['ko'].values)
eng_corpus = list(train['en'].values)

In [34]:
print("Korean:", ' '.join(kor_corpus[180]))
print()
print("English:", ' '.join(eng_corpus[180]))

Korean: 세균 의 그러 한 적응 능력 이 과학자 들 을 우려 하 게 하 고 있 다 .

English: <start> That adaptability worries scientists . <end>


# Step 3. 데이터 토큰화

앞서 정의한 tokenize() 함수를 사용해 데이터를 텐서로 변환하고 각각의 tokenizer를 얻으세요! 단어의 수는 실험을 통해 적당한 값을 맞춰주도록 합니다! (최소 10,000 이상!)

❗ 주의: 난이도에 비해 데이터가 많지 않아 훈련 데이터와 검증 데이터를 따로 나누지는 않습니다.

In [35]:
def tokenize(corpus, num_words=None):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, filters='')
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post') # pre

    return tensor, tokenizer

In [36]:
kor_tensor, kor_tokenizer = tokenize(kor_corpus, None)
eng_tensor, eng_tokenizer = tokenize(eng_corpus, None)

In [37]:
kor_tensor

array([[1117,  562,  371, ...,    0,    0,    0],
       [  48,  245,    5, ...,    0,    0,    0],
       [ 240,   36,   33, ...,  325,  103,    1],
       ...,
       [  67,    5, 1128, ...,    0,    0,    0],
       [  48,  377, 1124, ...,    0,    0,    0],
       [1032, 1048,   31, ...,    1,    0,    0]], dtype=int32)

In [38]:
print('단어 개수 :', kor_tokenizer.num_words)

단어 개수 : None


# Step 4. 모델 설계

한국어를 영어로 잘 번역해 줄 멋진 Attention 기반 Seq2seq 모델을 설계하세요! 앞서 만든 모델에 Dropout 모듈을 추가하면 성능이 더 좋아집니다! Embedding Size와 Hidden Size는 실험을 통해 적당한 값을 맞춰 주도록 합니다!

## BahdanauAttention

In [39]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units)
        self.w_enc = tf.keras.layers.Dense(units)
        self.w_com = tf.keras.layers.Dense(1)
    
    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]

        h_enc = self.w_enc(h_enc)
        h_dec = tf.expand_dims(h_dec, 1)
        h_dec = self.w_dec(h_dec)

        score = self.w_com(tf.nn.tanh(h_dec + h_enc))
        
        attn = tf.nn.softmax(score, axis=1)

        context_vec = attn * h_enc
        context_vec = tf.reduce_sum(context_vec, axis=1)

        return context_vec, attn

## Encoder

In [40]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(enc_units,
                                       return_sequences=True)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.gru(out)
        
        return out

## Decoder

In [41]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units,
                                       return_sequences=True,
                                       return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, h_dec, enc_out):
        context_vec, attn = self.attention(enc_out, h_dec)

        out = self.embedding(x)
        out = tf.concat([tf.expand_dims(context_vec, 1), out], axis=-1)
        
        out, h_dec = self.gru(out)
        out = tf.reshape(out, (-1, out.shape[2]))
        out = self.fc(out)

        return out, h_dec, attn

In [42]:
# 이전의 그래프와 세션 초기화
tf.keras.backend.clear_session()

In [43]:
# 코드를 실행하세요.

BATCH_SIZE     = 64
SRC_VOCAB_SIZE = len(kor_tokenizer.index_word) + 1
TGT_VOCAB_SIZE = len(eng_tokenizer.index_word) + 1

units         = 1024
embedding_dim = 512

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)

# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print ('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                     sample_state, sample_output)

print ('Decoder Output:', sample_logits.shape)
print ('Decoder Hidden State:', h_dec.shape)
print ('Attention:', attn.shape)

Encoder Output: (64, 30, 1024)
Decoder Output: (64, 14859)
Decoder Hidden State: (64, 1024)
Attention: (64, 30, 1)


# Step 5. 훈련하기

훈련엔 위에서 사용한 코드를 그대로 사용하되, eval_step() 부분이 없음에 유의합니다! 매 스텝 아래의 예문에 대한 번역을 생성하여 본인이 생각하기에 가장 멋지게 번역한 Case를 제출하세요! (Attention Map을 시각화해보는 것도 재밌을 거예요!)

❕ 참고: 데이터의 난이도가 높은 편이므로 생각만큼 결과가 잘 안나올 수 있습니다.

## Optimizer & Loss

In [44]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, # from_logits 는 확률 분포가 Softmax를 거쳐서 들어오는지, 모델의 출력값 그대로 들어오는지를 결정합니다. 우리는 True 로 줬으니 모델의 출력값을 그대로 전달하면 됩니다!
    reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    
    return tf.reduce_mean(loss)

## train_step()

In [45]:
# @tf.function(데코레이터)는 훈련 외적인 텐서플로우 연산을 GPU에서 동작하게 해 훈련을 가속할 수 있도록 도와줍니다

@tf.function
def train_step(src, tgt, encoder, decoder, optimizer, dec_tok):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape:
        enc_out = encoder(src)
        h_dec = enc_out[:, -1]
        
        dec_src = tf.expand_dims([dec_tok.word_index['<start>']] * bsz, 1)

        for t in range(1, tgt.shape[1]):
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

            loss += loss_function(tgt[:, t], pred)
            dec_src = tf.expand_dims(tgt[:, t], 1)
        
    batch_loss = (loss / int(tgt.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

## Train

In [ ]:
EPOCHS = 100

be_total_loss = 99999
for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, kor_tensor.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)
    for (batch, idx) in enumerate(t):
        batch_loss = train_step(kor_tensor[idx:idx+BATCH_SIZE],
                                eng_tensor[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                eng_tokenizer)
        
        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))
    
    
    if be_total_loss < (total_loss / (batch + 1)):
        break
    else:
        be_total_loss = (total_loss / (batch + 1))

Epoch 21:  92%|█████████▏| 193/210 [00:35<00:03,  5.40it/s, Loss 1.2757]

- 메모리 문제
    - 데이터 축소 : 약 6만개에서 3만개로 축소
    - 토크나이저 단어개수 : 1만개
    - 배치 축소 : 64 에서 8

- 학습 속도(1ep)
    - 데이터 개수 : 약 50,000개(전체) 일 때, 1시간 13분
    - 데이터 개수 : 30,000개 일 때, 30분
    - 데이터 개수 : 10,000개 일 때, 18분
    - 데이터 개수 : 5,000개 일 때, 2분
    - 데이터 개수 : 1,000개 일 때, 20초

### 모델 저장 & 불러오기

In [125]:
# # 모델 저장
# encoder.save_weights('/aiffel/aiffel/s2s_translation/weights/encoder_weights_50000_ep1')
# decoder.save_weights('/aiffel/aiffel/s2s_translation/weights/decoder_weights_50000_ep1')


In [26]:
# # 모델 구조를 불러오고 가중치를 로드하여 모델을 복원
# loaded_encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
# loaded_encoder.load_weights('/aiffel/aiffel/s2s_translation/weights/encoder_weights2')

# loaded_decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)
# loaded_decoder.load_weights('/aiffel/aiffel/s2s_translation/weights/decoder_weights2')


## evaluate

In [23]:
def preprocessing_eval(txt):
    sentence = txt.strip()                                         # 문장의 양쪽 공백 제거
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)                   # 특수 문자 및 구두점 주변에 공백 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                         # 여러 개의 공백을 하나의 공백으로 대체
    sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z?.!,]+", " ", sentence)  # 한글 및 영어 이외의 문자는 공백으로 대체
    sentence = sentence.strip()                                         # 다시 양쪽 공백 제거

    return sentence

In [24]:
def evaluate(sentence, encoder, decoder):
    attention = np.zeros((eng_tensor.shape[-1], kor_tensor.shape[-1]))
    
    sentence = preprocessing_eval(sentence)
    inputs = kor_tokenizer.texts_to_sequences([sentence.split()])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                           maxlen=kor_tensor.shape[-1],
                                                           padding='post')

    result = ''

    enc_out = encoder(inputs)

    dec_hidden = enc_out[:, -1]
    dec_input = tf.expand_dims([eng_tokenizer.word_index['<start>']], 0)

    for t in range(eng_tensor.shape[-1]):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention[t] = attention_weights.numpy()

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0]).numpy()

        result += eng_tokenizer.index_word[predicted_id] + ' '

        if eng_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention

import seaborn as sns
sns.set(font='NanumGothic')

def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    ax.set_xticklabels([''] + sentence, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)

    print('Input: %s' % (sentence))
    print('Output: {}'.format(result))
    
    attention = attention[:len(result.split()), :len(sentence.split())]
    #plot_attention(attention, sentence.split(), result.split(' '))

'''
K1) 오바마는 대통령이다.
K2) 시민들은 도시 속에 산다.
K3) 커피는 필요 없다.
K4) 일곱 명의 사망자가 발생했다.
'''
translate("오바마는 대통령이다.", encoder, decoder)
translate("시민들은 도시 속에 산다.", encoder, decoder)
translate("커피는 필요 없다.", encoder, decoder)
translate("일곱 명의 사망자가 발생했다.", encoder, decoder)

Input: 오바마는 대통령이다 .
Output: . <end> 
Input: 시민들은 도시 속에 산다 .
Output: i m confident that s native of the game of the game of the game of the game of the 
Input: 커피는 필요 없다 .
Output: i think of my career <end> 
Input: 일곱 명의 사망자가 발생했다 .
Output: the whale . <end> 


# 보고서

## 실험 조건

1. 데이터 개수 : 1000 / 5000 / 30000 / 50000(전체)
2. 토큰 길이 : 40 / 20 
3. 단어장 개수 : 10000 / All
4. total_loss(또는 에폭 loss)가 1번 이라도 감소하지 않으면 멈추게 설정

## 결론

> 아래 실험 결과 토대로 작성함

1. epoch이 증가함에 따라 loss는 감소하지만, 성능은 떨어진다 ( 토큰 길이 40개 이하, 데이터 개수 1000개 에폭 10 과 23 비교 )
2. 데이터 개수가 증가함에 따라 성능이 향상됨은 아니다 ( 물론, 데이터가 증가함에 따라 epoch을 늘려야하지만, 학습 시간이 오래 걸림에 따라 실험 못해봄)
3. 

# 실험 결과

- 토큰 길이 40개 이하

|데이터 개수|ep| 예문 | 제출 | 파파고 |
|:---:|:---:|:---|:---|:---|
|1000|10| 오바마는 대통령이다. | the faithful .| 신실한 사람들. |
||| 시민들은 도시 속에 산다. | the glass and in the government is a lot of the u .| 유리와 정부에 있는 많은 사람들은 당신입니다. |
||| 커피는 필요 없다. | they are about .| 그들은 대략. | 
||| 일곱 명의 사망자가 발생했다. | the government is a few .| 정부는 소수입니다.| 
|1000|23| 오바마는 대통령이다. | there|  |
||| 시민들은 도시 속에 산다. | urban |  |
||| 커피는 필요 없다. | the dawn .|  | 
||| 일곱 명의 사망자가 발생했다. |the other these business . , there , the other these different argument ....| | 
|5000|10| 오바마는 대통령이다. | the first of the first of the first...|  |
||| 시민들은 도시 속에 산다. | the north korea s .|  |
||| 커피는 필요 없다. | but the first of the first but the first of the first but the first of the first|  | 
||| 일곱 명의 사망자가 발생했다. | the first of the first of the first of the first of the first of the first of| | 
|50000| 1 |  오바마는 대통령이다. | the , the , the the , the , the the , the , the the , the , the |  |
||| 시민들은 도시 속에 산다. | the internet . |  |
||| 커피는 필요 없다. | the , the , the the , the , the the , the , the the , the , the |  | 
||| 일곱 명의 사망자가 발생했다. | the , the , the the , the , the the , the , the the , the , the | | 

- 토큰 길이 20개 이하(학습 속도가 느려서 줄임) + 단어장 개수 10,000개

|데이터 개수|ep| 예문 | 제출 | 파파고 |
|:---:|:---:|:---|:---|:---|
|13000|10| 오바마는 대통령이다. | .|  |
||| 시민들은 도시 속에 산다. |the red sox are also demanding to the divide between the divide between the divide between the divide between the  |  |
||| 커피는 필요 없다. |i can be honored |  | 
||| 일곱 명의 사망자가 발생했다. |the grief stricken crowd spilled outside | | 

- 토큰 길이 20개 이하(학습 속도가 느려서 줄임) + 단어장 개수 ALL

|데이터 개수|ep| 예문 | 제출 | 파파고 |
|:---:|:---:|:---|:---|:---|
|13000|10| 오바마는 대통령이다. |. |  |
||| 시민들은 도시 속에 산다. | i m confident that s native of the game of the game of the game of the game of the |  |
||| 커피는 필요 없다. | i think of my career|  | 
||| 일곱 명의 사망자가 발생했다. | the whale| | 

|Num|평가문항	|상세기준|
|:---:|:---|:---|
|1| 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 한국어 포함하여 잘 이루어졌다.|	구두점, 대소문자, 띄어쓰기, 한글 형태소분석 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.|
|2| Attentional Seq2seq 모델이 정상적으로 구동된다.|	seq2seq 모델 훈련 과정에서 training loss가 안정적으로 떨어지면서 학습이 진행됨이 확인되었다.|
|3|테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.|	테스트용 디코더 모델이 정상적으로 만들어져서, 정답과 어느 정도 유사한 영어 번역이 진행됨을 확인하였다.|

[공부에 도움되는 사이트](http://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/)